#  Additional Cleanup

Now that I have all my lyrics I need to clean up obscure characters such as \n & \ from the strings. I will also remove the [verse 1] [chorus] parts because the artist's names are listed in there and will throw my model off.Finally, I will create a binarized column on whether the artist self-announces at the beginning of the track, I will need to extract the &feat artist name from title to add to the artist column.

Because of the time it took to gather my data, I will be doing cleanup in 3 similar notebooks, as data becomes available. All 3 notebooks will look like this.

In [77]:
import pandas as pd
import numpy as np
import regex as re

import lyricsgenius
from requests.exceptions import Timeout

pd.options.display.max_colwidth = 60
%load_ext email_notify_magic

The email_notify_magic extension is already loaded. To reload it, use:
  %reload_ext email_notify_magic


In [2]:
df = pd.read_csv('./data/lyrics_cp.csv', index_col=0)

In [3]:
df.head()

,artists,title,track_id,popularity,explicit,release_year,lyrics
0,Shawn Mendes,Wonder,5KCbr5ndeby4y4ggthdiAb,0,False,2020,"[Verse 1]\nI wonder if I'm being real\nDo I speak my truth or do I filter how I feel?\nI wonder, wouldn't it be nice\nTo live inside a world that isn't black and white?\nI wonder what it's like to be my friends\nHope that they don't think I forget about them\nI wonder, I wonder\n\n[Chorus]\nRight before I close my eyes\nThe only thing that's on my mind\nBeen dreaming that you feel it, too\nI w..."
1,Justin Bieber,Holy (feat. Chance The Rapper),5u1n1kITHCxxp8twBcZxWy,92,False,2020,Holy (feat. Chance The Rapper) - Justin Bieber\nDiamonds - Sam Smith\nMe Gusta (with Cardi B & Myke Towers) - Anitta\nMood Swings (feat. Lil Tjay & Summer Walker) - Remix - Pop Smoke\nWelcome Back (feat. Alessia Cara) - Ali Gatie\nMe x 7 (feat. Tierra Whack) - Alicia Keys\nGates to the Sun (POLLEN Singles) - SahBabii\nSaid Sum (feat. City Girls & DaBaby) - Remix - Moneybagg Yo\nDolly (with Lil...
2,24kGoldn,Mood (feat. Iann Dior),3tjFYV6RSFtuktYl3ZtYcq,100,True,2020,1. 24kGoldn- Mood (feat. iann dior)\n2. 24kGoldn- CITY OF ANGELS\n3. Clean Bandit & Mabel- Tick Tock (feat. 24kGoldn)\n4. DVBBS- Tinted Eyes (feat. blackbear & 24kGoldn)\n5. Landon Cube- Pretty (feat. 24kGoldn)\n6. 24kGoldn- VALENTINO\n7. Landon Cube- Eighties (feat. 24kGoldn)\n8. Internet Money- Giddy Up (feat. Wiz Khalifa & 24kGoldn)\n9. 24kGoldn- Unbelievable (feat. Kaash Paige)\n10. 24kGol...
3,Internet Money,Lemonade,02kDW379Yfd5PzW5A6vuGt,93,True,2020,"[Pre-Chorus: Don Toliver]\nXanny bars, suicide door, brand new bag\nCollege girls give a nigga head in my Rafs\nRockstar life, so much money it'll make you laugh, hey\nThese bitches, they hate, and you can't miss what you never had, hey, hey\n\n[Chorus: Don Toliver]\nOff the juice (Juice), codeine got me trippin' (Juice)\nCopped the coupe (Coupe), woke up, roof is missin' (Yeah)\nIce (Ice), le..."
4,BLACKPINK,Bet You Wanna (feat. Cardi B),1hPkiovjTqiJAJen4uyNRg,0,False,2020,Wonder - Shawn Mendes\nBet You Wanna (feat. Cardi B) - BLACKPINK\nRunnin - 21 Savage\nDon’t Stop (feat. Young Thug) - Megan Thee Stallion\nOutta Time (feat. Drake) - Bryson Tiller\nNot Another Love Song - Ella Mai\nLie Like This - Julia Michaels\nFallin' - Why Don't We\nheart won't let me - LANY\nSavage Love (Laxed – Siren Beat) [BTS Remix] - Jawsh 685\nJeanie (feat. Bon Iver) - Jim-E Stack\nS...


Making sure there are no duplicates of artist and title again.

In [4]:
df.duplicated(['artists', 'title']).value_counts()

False    18116
dtype: int64

In making sure the lyrics were added correctly I noticed that for all songs that have (feat. artist) in the song title, the lyrics did not get pulled properly.
I will call lyricsgenius again to fix this.

In [5]:
df[df['title'].str.contains("feat.")]

,artists,title,track_id,popularity,explicit,release_year,lyrics
1,Justin Bieber,Holy (feat. Chance The Rapper),5u1n1kITHCxxp8twBcZxWy,92,False,2020,Holy (feat. Chance The Rapper) - Justin Bieber\nDiamonds - Sam Smith\nMe Gusta (with Cardi B & Myke Towers) - Anitta\nMood Swings (feat. Lil Tjay & Summer Walker) - Remix - Pop Smoke\nWelcome Back (feat. Alessia Cara) - Ali Gatie\nMe x 7 (feat. Tierra Whack) - Alicia Keys\nGates to the Sun (POLLEN Singles) - SahBabii\nSaid Sum (feat. City Girls & DaBaby) - Remix - Moneybagg Yo\nDolly (with Lil...
2,24kGoldn,Mood (feat. Iann Dior),3tjFYV6RSFtuktYl3ZtYcq,100,True,2020,1. 24kGoldn- Mood (feat. iann dior)\n2. 24kGoldn- CITY OF ANGELS\n3. Clean Bandit & Mabel- Tick Tock (feat. 24kGoldn)\n4. DVBBS- Tinted Eyes (feat. blackbear & 24kGoldn)\n5. Landon Cube- Pretty (feat. 24kGoldn)\n6. 24kGoldn- VALENTINO\n7. Landon Cube- Eighties (feat. 24kGoldn)\n8. Internet Money- Giddy Up (feat. Wiz Khalifa & 24kGoldn)\n9. 24kGoldn- Unbelievable (feat. Kaash Paige)\n10. 24kGol...
4,BLACKPINK,Bet You Wanna (feat. Cardi B),1hPkiovjTqiJAJen4uyNRg,0,False,2020,Wonder - Shawn Mendes\nBet You Wanna (feat. Cardi B) - BLACKPINK\nRunnin - 21 Savage\nDon’t Stop (feat. Young Thug) - Megan Thee Stallion\nOutta Time (feat. Drake) - Bryson Tiller\nNot Another Love Song - Ella Mai\nLie Like This - Julia Michaels\nFallin' - Why Don't We\nheart won't let me - LANY\nSavage Love (Laxed – Siren Beat) [BTS Remix] - Jawsh 685\nJeanie (feat. Bon Iver) - Jim-E Stack\nS...
5,Joel Corry,Head & Heart (feat. MNEK),4wosxLl0mAqhneDzya2MfY,76,False,2020,1. Tata McRae- You Broke Me First\n2. 24kGoldn- Mood (feat. iann dior)\n3. BTS- Dynamite\n4. Cardi B- WAP (feat. Megan Thee Stallion)\n5. DaBaby- ROCKSTAR (feat. Roddy Ricch)\n6. Joel Corry- Head & Heart (feat. MNEK)\n7. Topic & A7S- Breaking Me\n8. Harry Styles- Watermelon Sugar\n9. Drake- Laugh Now Cry Later (feat. Lil Durk)\n10. Miley Cyrus- Midnight Sky\n11. BLACKPINK & Selena Gomez- Ice C...
8,Pop Smoke,For The Night (feat. Lil Baby & DaBaby),0PvFJmanyNQMseIFrU708S,95,True,2020,"1. Pop Smoke- For The Night (feat. Lil Baby & DaBaby)\n2. Pop Smoke- Dior\n3. Pop Smoke- Hello (feat. A Boogie wit Da Hoodie)\n4. Pop Smoke- Get Back\n5. Pop Smoke- Mood Swings (feat. Lil Tjay)\n6. JACKBOYS, Pop Smoke & Travis Scott- GATTI\n7. H.E.R.- Slide (Remix) (feat. Pop Smoke, A Boogie wit Da Hoodie & Chris Brown)\n8. Pop Smoke- Element\n9. Pop Smoke- The Woo (feat. 50 Cent & Roddy Ricch..."
...,...,...,...,...,...,...,...
20458,Justin Bieber,Forever (feat. Post Malone & Clever),2ZlCGeK30BLRNSPC832pNZ,76,False,2020,"Billie Eilish - No Time To Die\nJustin Bieber - Forever (feat. Post Malone & Clever)\nSam Smith - To Die For\nA Boogie Wit da Hoodie - Numbers (feat. Roddy Ricch, Gunna and London On Da Track)\nMigos - Give No Fxk\nTame Impala - Breathe Deeper\nAlec Benjamin - Oh My God\nJustin Bieber - Second Emotion (feat. Travis Scott)\nYG - Konclusions\nBad Bunny - Ignorantes\nMadison Beer - Selfish\nPowfu..."
20463,Chris Brown,Next To You (feat. Justin Bieber),2faDo9tBRN57TN4iK2XFWt,0,False,2011,2005Whose Girl Is That 👕\nRun It! (feat. Juelz Santana) 👕\nYo (Excuse Me Miss) 👕\n2006Gimme That (feat. Lil Wayne) 👕\nSay Goodbye 👕\nShortie Like Mine (Bow Wow feat. Chris Brown & Johntá Austin) 👕\n2007Wall to Wall 👕\nKiss Kiss (feat. T-Pain) 👕\nThis Christmas 👕\nWith You 👕\n2008Shawty Get Loose (Lil Mama feat. Chris Brown & T-Pain) 👕\nNo Air (Jordin Sparks feat. Chris Brown) 👕\nTake You Down ...
20475,Justin Bieber,Second Emotion (feat. Travis Scott),1PnAx788sAvcI3ZxTlEyX8,66,False,2020,"Billie Eilish - No Time To Die\nJustin Bieber - Forever (feat. Post Malone & Clever)\nSam Smith - To Die For\nA Boogie Wit da Hoodie - Numbers (feat. Roddy Ricch, Gunna and London On Da Track)\nMigos - Give No Fxk\nTame Impala - Breathe Deeper\nAlec Benjamin - Oh My God\nJustin Bieber - Second Emotion (feat. Travis Scott)\nYG - Konclusions\nBad Bunny - Ignorantes\nMadison Beer - Selfish\nPowfu..."
20482,Justin Bieber,Runnin

In [6]:
feat = []    
for values in df['title']:
    try:
        search = re.search(r'(?:\(|\[)(?:feat.|feat.|feat |with )[^)]*(?:\)|\])', values, re.I).group()
        feat.append(search.replace('(' or '[', '').replace('feat. ' or 'Feat. ' or 'with ', '').replace(')' or ']', '')) 

    except:
        feat.append('')
        
    
df['title'] = df['title'].str.replace(r'(?:\(|\[)(?:feat. |feat |with )[^)]*(?:\)|\])','', re.I)
df.insert(1, 'featuring', feat) 
#https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns?page=1&tab=votes#tab-top
# Thanks Brett for Regex Help

In [131]:
df.head(3)

,artists,featuring,title,track_id,popularity,explicit,release_year,lyricss,lyrics
0,Shawn Mendes,,Wonder,5KCbr5ndeby4y4ggthdiAb,0,False,2020,[Verse 1]\nI wonder if I'm being real\nDo I speak my tru...,NaN
1,Justin Bieber,Chance The Rapper,Holy,5u1n1kITHCxxp8twBcZxWy,92,False,2020,[Verse 1: Justin Bieber]\nI hear a lot about sinners\nDo...,NaN
2,24kGoldn,Iann Dior,Mood,3tjFYV6RSFtuktYl3ZtYcq,100,True,2020,"[Intro: 24kGoldn]\nOh-oh-oh\nYeah, yeah, yeah, yeah (Yea...",NaN


I'm just going to manually fix the rest of these, some didn't have brackets around them and for some reason they just aren't moving over.

In [8]:
df[df['title'].str.contains("feat")].head()

,artists,featuring,title,track_id,popularity,explicit,release_year,lyrics
33,Kane Brown,,Be Like That - feat. Swae Lee & Khalid,5f1joOtoMeyppIcJGZQvqJ,87,False,2020,1. Tata McRae- You Broke Me First\n2. 24kGoldn- Mood (feat. iann dior)\n3. BTS- Dynamite\n4. Cardi B- WAP (feat. Megan Thee Stallion)\n5. DaBaby- ROCKSTAR (feat. Roddy Ricch)\n6. Joel Corry- Head & Heart (feat. MNEK)\n7. Topic & A7S- Breaking Me\n8. Harry Styles- Watermelon Sugar\n9. Drake- Laugh Now Cry Later (feat. Lil Durk)\n10. Miley Cyrus- Midnight Sky\n11. BLACKPINK & Selena Gomez- Ice C...
154,HIXTAPE,,"One Beer (HARDY feat. Lauren Alaina, Devin Dawson)",5FmvaZGd6fulojSToozLRD,77,False,2019,NaN
281,Smoke DZA,,Boatloads feat. Jack Harlow,7z6LjB2BTrH9MTMPMGSYOu,0,True,2020,Wonder - Shawn Mendes\nBet You Wanna (feat. Cardi B) - BLACKPINK\nRunnin - 21 Savage\nDon’t Stop (feat. Young Thug) - Megan Thee Stallion\nOutta Time (feat. Drake) - Bryson Tiller\nNot Another Love Song - Ella Mai\nLie Like This - Julia Michaels\nFallin' - Why Don't We\nheart won't let me - LANY\nSavage Love (Laxed – Siren Beat) [BTS Remix] - Jawsh 685\nJeanie (feat. Bon Iver) - Jim-E Stack\nS...
1929,Bino Rideaux,,Mismatch (The Remix) feat. Young Thug,3MP8ItylVgtl5ZzDfb16jY,57,True,2020,NaN
1971,The LOX,,Do To Me feat. Jeremih,098FfxMmKJaAcJnAyxmNG4,52,True,2020,(*Background Vocals*)\n[Chorus: Jeremih]\nI blow her little\nThat's what you do to me\nIt made my baby mama lurk\nThat's what you do to me\nLet me take off the work\nThat's what you do to me\nLet me take off my shit\nThat's what you do to me\nTold my niggas money business\nThat's what you do to me\nFirst class to Bahamas\nThat's what you do to me\n[?] not do bitches\nThat's what you do to me\n...


In [9]:
def feat_title(track_id, feat, title):
    df.loc[df['track_id'] == track_id, ['featuring', 'title']] = [feat,title]

In [10]:
feat_title('5f1joOtoMeyppIcJGZQvqJ', 'Swae Lee & Khalid', 'Be Like That')
df.loc[df['track_id'] == '5FmvaZGd6fulojSToozLRD', ['artists']] = ['HARDY']
feat_title('5FmvaZGd6fulojSToozLRD', 'Lauren Alaina & Devin Dawson', 'One Beer')
feat_title('7z6LjB2BTrH9MTMPMGSYOu', 'Jack Harlow', 'Boatloads')
feat_title('3MP8ItylVgtl5ZzDfb16jY', 'Young Thug', 'Mismatch (The Remix)')
feat_title('098FfxMmKJaAcJnAyxmNG4', 'Jeremih', 'Do To Me')
feat_title('0gmUqyJeW8o9Gc4KnQgZdT', 'Lil Gotit', 'Hood Baby – Remix')
feat_title('0gO3E51A3fqp0ypJ26QbVP', 'Lil Yachty & Tierra Whack feat. A$AP Rocky & Tyler, The Creator', 'T.D')
feat_title('213lc71WFDmKgHDFhR55rP', 'Sons of Sonix', 'plenty')
feat_title('5Kr01FtRG1qf7nHJJoGAX9', 'Daniela Andrade', 'Hold')
feat_title('6zN6wjHO3ALVCIQQhSqqqN', 'Thomas Rhett & Florida Georgia Line', 'Thank You Lord')
feat_title('7rNjupJkjsc3ANGeW0RCYj', 'Buddy', 'Teen Scene')
feat_title('1FRJv6T0DTGsfVhGkusDN5', 'Deem Spencer', 'SUNRISE')
feat_title('1HazDXnvOTubBFmZnabq7Q', 'Anthony Hamilton', 'So In Love')
feat_title('65sJlRR3cK1WUP7yC15J6R', 'Micah Martin', 'Lose Control')
feat_title('3huk8B1aNp9txmknHOzsq2', 'Joy Tyson', 'Hella')
feat_title('5G15fXRiD0y7pgkoDRrkRV', 'Radwan Ghazi Moumneh ', 'In My Heart')
feat_title('4EsFXXYwZYh4GD37aff1Db', 'Shift K3Y', 'Better Off Without You')
feat_title('3nQLd23SbGvRd6lpdzWAxf', 'Carasel', 'FIRE')
feat_title('5gtpmyiI4giWRZrPD3E4HU', 'Tomas Høffding', 'Bloodline')
feat_title('3Vo4wInECJQuz9BIBMOu8i', 'Cardi B', 'Finesse - Remix')

Stripping spaces at the beginning and end of the column values. I manually filled in some values and used regex and .replace the likeliness of unwanted spaces is high. Making sure that it will not be a factor in pulling my data.

In [19]:
df['artists'] = df['artists'].str.strip()
df['featuring'] = df['featuring'].str.strip()
df['title'] = df['title'].str.strip()

### Re-Filling Missing / Misfilled lyrics
Some rows did not get any lyrics pulled or the 'lyrics' were a list of other collaborations,  because the feat. in the title was throwing the API request off. I will now refill those rows.

In [20]:
def get_lyrics(): # no arguments needed
    genius = lyricsgenius.Genius("2AaY5B5Iylvf3X2oQl_4Jmk74pSZ-J8VqcboMk1PO_y7N8wlN83D9a85uhNh49Id") # token
    genius.timeout = 15  #timeout
    for track in df.values: # for each row in the dataframe
        if track[1] != '': #if featuring column is not empty
            retries = 0 # retry counter
            while retries < 3: # while retries are less than 3
                try:
                    song = genius.search_song(track[2], track[0]) # get song based on title and artist
                    if song is not None:
                        df.loc[df['featuring'] == track[1], 'lyrics'] = song.lyrics # using .loc at feature column location replace the lyric with gathered lyrics
                    else:
                        df.loc[df['featuring'] == track[1], 'lyrics'] = np.NaN # replace the lyric with NaN if nothing is gathered
                    break # break to go to next track
                except Timeout as e: # if timeout
                    retries += 1 # add to counter and try again
                    continue
        else:
            pass # if featuring column is an empty string, pass.

In [25]:
get_lyrics()

This was my first set pulled with the API, in checking the lyrics duplicates, I noticed some lyrics didn't get pulled properly, duplicated, and a mess. I repulled all 18k rows with the featuring artist separated, and got better results. I'll continue below with the corrected information

In [64]:
fixed = pd.read_csv('./data/corrected_lyrics_cp1', index_col = 0)
fixed.rename(columns = {'artists':'artist', 'lyricss':'lyrics'}, inplace=True)

In [65]:
fixed.head()

,artist,featuring,title,track_id,popularity,explicit,release_year,lyrics
0,Shawn Mendes,NaN,Wonder,5KCbr5ndeby4y4ggthdiAb,0,False,2020,"[Verse 1]\nI wonder if I'm being real\nDo I speak my truth or do I filter how I feel?\nI wonder, wouldn't it be nice\nTo live inside a world that isn't black and white?\nI wonder what it's like to be my friends\nHope that they don't think I forget about them\nI wonder, I wonder\n\n[Chorus]\nRight before I close my eyes\nThe only thing that's on my mind\nBeen dreaming that you feel it, too\nI w..."
1,Justin Bieber,Chance The Rapper,Holy,5u1n1kITHCxxp8twBcZxWy,92,False,2020,"[Verse 1: Justin Bieber]\nI hear a lot about sinners\nDon't think that I'll be a saint\nBut I might go down to the river\n'Cause the way that the sky opens up when we touch\nYeah, it's making me say\n\n[Chorus: Justin Bieber]\nThat the way you hold me, hold me, hold me, hold me, hold me\nFeels so holy, holy, holy, holy, holy\nOn God\nRunnin' to the altar like a track star\nCan't wait another s..."
2,24kGoldn,Iann Dior,Mood,3tjFYV6RSFtuktYl3ZtYcq,100,True,2020,"[Intro: 24kGoldn]\nOh-oh-oh\nYeah, yeah, yeah, yeah (Yeah)\n\n[Chorus: 24kGoldn]\nWhy you always in a mood? Fuckin 'round, actin' brand new\nI ain't tryna tell you what to do, but try to play it cool\nBaby, I ain't playing by your rules\nEverything look better with a view\nWhy you always in a mood? Fuckin 'round, actin' brand new\nI ain't tryna tell you what to do, but try to play it cool\nBab..."
3,Internet Money,NaN,Lemonade,02kDW379Yfd5PzW5A6vuGt,93,True,2020,"[Pre-Chorus: Don Toliver]\nXanny bars, suicide door, brand new bag\nCollege girls give a nigga head in my Rafs\nRockstar life, so much money it'll make you laugh, hey\nThese bitches, they hate, and you can't miss what you never had, hey, hey\n\n[Chorus: Don Toliver]\nOff the juice (Juice), codeine got me trippin' (Juice)\nCopped the coupe (Coupe), woke up, roof is missin' (Yeah)\nIce (Ice), le..."
4,BLACKPINK,Cardi B,Bet You Wanna,1hPkiovjTqiJAJen4uyNRg,0,False,2020,"[Intro: Cardi B]\nBLACKPINK\nCardi\n\n[Verse 1: Rosé, Lisa, Jisoo]\nTell me where you wanna go\nI'll meet you with my bags at the door\nUh, I'm gonna make you go blind\nEvery time I walk, my hips, they don't lie\nTake me to your paradise\n'Cause I don't wanna wait anymore\nUh, I'll say it one more time\nEvery time I walk, my hips, they don't lie\n\n[Pre-Chorus: Jennie & Rosé]\nYou wanna touch,..."


In [81]:
pd.set_option('display.max_rows', 10)

In [163]:
fixed[fixed.duplicated('lyrics', keep = False)].sort_values('lyrics').head(20)

,artist,featuring,title,track_id,popularity,explicit,release_year,lyrics
18938,The Beatles,NaN,Penny Lane - Remastered,5RStjc42UAYI2NMY3cYpgz,0,False,1967,"""I promis'd you an account of what befel me, and am now ..."
18996,The Beatles,NaN,The Long And Winding Road - Remastered,2rG2c14sIgzyIRdbw3edaL,0,False,1970,"""I promis'd you an account of what befel me, and am now ..."
19224,The Beatles,NaN,Dig A Pony - Remastered,3eMeNJhwxiecXnSYy2NhfY,0,False,1970,"""I promis'd you an account of what befel me, and am now ..."
18941,The Beatles,NaN,Oh! Darling - Remastered,59CLXQLZKxRPzMW8S8Gt9N,0,False,1969,"""I promis'd you an account of what befel me, and am now ..."
780,Sumsar,NaN,Amorous Rebound,0JYn3juUc1lZ4kFeGey1kI,60,False,2019,"""O lamentable ruins of the ill-fated Nicosia,[76] still ..."
...,...,...,...,...,...,...,...,...
18952,The Beatles,NaN,Can't Buy Me Love - Remastered,7KmRz1oKEwgj9qpfn72OfO,0,False,1964,"""The schools in a portion of Boston stretching from just..."
19093,The Beatles,NaN,Because - Remastered,4cLoL5KPfE1hAwfsO84FX7,0,False,1969,"""The schools in a portion of Boston stretching from just..."
19018,The Beatles,NaN,I Wanna Be Your Man - Remastered,2aibmsfQ8WFl2AfCkNi5RG,0,False,1963,"""The schools in a portion of Boston stretching from just..."
9753,Zeni N,NaN,Still Together - Radio Mix,4onD0pjASg29nhthK87ZQb,44,False,2018,"""The schools in a portion of Boston stretching from just..."


It looks like the - Remastered or -Live or any of this variation is also throwing some of these fetches off. I'll remove those and refetch. Others that were marked as duplicates, are songs that genius doesn't have lyrics for so this API library just pulled a variation of the 'All Results' page.

In [134]:
dups = fixed[fixed.duplicated('lyrics', keep = False)]

In [135]:
dups['title'] = dups['title'].str.replace(r'-[^-]*$','', re.I)

<ipython-input-135-155eaf6d1b92>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dups['title'] = dups['title'].str.replace(r'-[^-]*$','', re.I)


In [136]:
fixed.loc[fixed['track_id'] == '5sBplhIrmzA4kQGyIOpmDf', ['featuring', 'title']] = ['Roddy Ricch & Don Toliver', 'Lemonade - Remix']

In [114]:
def get_lyrics():
    genius = lyricsgenius.Genius("2AaY5B5Iylvf3X2oQl_4Jmk74pSZ-J8VqcboMk1PO_y7N8wlN83D9a85uhNh49Id")
    genius.timeout = 15
    for track in df.values:
        for dup in dups.values:
            if track[3] == dup[3] :
                retries = 0
                while retries < 3:
                    try:
                        song = genius.search_song(dup[2], dup[0])
                        if song is not None:
                            df.loc[df['track_id'] == track[3], 'lyrics'] = song.lyrics
                        else:
                            df.loc[df['track_id'] == track[3], 'lyrics'] = np.NaN
                        break
                    except Timeout as e:
                        retries += 1
                        continue
            else:
                pass

In [115]:
get_lyrics()

Searching for "Lemonade - Remix" by Internet Money...
Done.
Searching for "Tap In" by Saweetie...
Done.
Searching for "Nothing's Better" by Filmore...
Specified song does not contain lyrics. Rejecting.
Searching for "State I'm In" by Filmore...
Specified song does not contain lyrics. Rejecting.
Searching for "Lovin' On You" by Luke Combs...
Done.
Searching for "Things I Can't Say" by Spencer Crandall...
Specified song does not contain lyrics. Rejecting.
Searching for "Tattoo - Remix with Camilo" by Rauw Alejandro...
No results found for: 'Tattoo - Remix with Camilo Rauw Alejandro'
Searching for "Agua  - Music From "Sponge On The Run" Movie" by Tainy...
No results found for: 'Agua  - Music From "Sponge On The Run" Movie Tainy'
Searching for "Salt Licorice  - Radio Edit" by Jónsi...
Done.
Searching for "1,000,000 X Better" by Griff...
Done.
Searching for "Somersault" by Lourdiz...
No results found for: 'Somersault Lourdiz'
Searching for "Yessirski" by Doe Boy...
Specified song does not c

Done.
Searching for "What The Rain Whispered Softly" by Jef Martens...
No results found for: 'What The Rain Whispered Softly Jef Martens'
Searching for "Whitby" by Stephan Moccio...
Done.
Searching for "Synchronicity" by Allysium...
No results found for: 'Synchronicity Allysium'
Searching for "Le Castellet" by Antoine Minot...
Done.
Searching for "Des ponts plutôt que des murs" by Zdenek Lemelin...
No results found for: 'Des ponts plutôt que des murs Zdenek Lemelin'
Searching for "Moment of Silence" by Edvard Kravchuk...
No results found for: 'Moment of Silence Edvard Kravchuk'
Searching for "Love Will Set Us Free" by Novum Cordis...
Done.
Searching for "When She Flows" by Jef Martens...
No results found for: 'When She Flows Jef Martens'
Searching for "United" by Cory Mickens...
Done.
Searching for "Maru" by Hideyuki Hashimoto...
No results found for: 'Maru Hideyuki Hashimoto'
Searching for "Lacrimosa" by Lucas Vendrai...
Done.
Searching for "Stretch of Clouds" by Taylor Driskell...
Do

No results found for: 'Weaving Moy Finneas'
Searching for "Faller Snö" by Matti Bye...
No results found for: 'Faller Snö Matti Bye'
Searching for "Theme From the Lost Ones" by Justin Withers...
Done.
Searching for "Zakopane" by Zygmunt Czarnecki...
No results found for: 'Zakopane Zygmunt Czarnecki'
Searching for "Prends soin de toi" by Falerina Paquet...
No results found for: 'Prends soin de toi Falerina Paquet'
Searching for "Kjæreste" by Alida Sørby...
No results found for: 'Kjæreste Alida Sørby'
Searching for "supine" by Teodor Wolgers...
Done.
Searching for "Fairfield" by Isaac Nash...
No results found for: 'Fairfield Isaac Nash'
Searching for "Uitwaaien" by Juultje Berben...
No results found for: 'Uitwaaien Juultje Berben'
Searching for "Den första snön" by Maya Åström...
No results found for: 'Den första snön Maya Åström'
Searching for "Silent Agreement" by Joel Lyssarides...
No results found for: 'Silent Agreement Joel Lyssarides'
Searching for "Súlur" by Petra Birgisdóttir...
N

Done.
Searching for "Father Figure - Remastered" by George Michael...
Done.
Searching for "I Can't Dance - 2007 Digital Remaster" by Genesis...
Done.
Searching for "My Heart Will Go On - Love Theme from "Titanic"" by Céline Dion...
Done.
Searching for "Men In Black - From "Men In Black" Soundtrack" by Will Smith...
Done.
Searching for "I Still Haven't Found What I'm Looking For - Remastered 2007" by U2...
No results found for: 'I Still Haven't Found What I'm Looking For - Remastered 2007 U2'
Searching for "The Heat Is On - From "Beverly Hills Cop" Soundtrack" by Glenn Frey...
No results found for: 'The Heat Is On - From "Beverly Hills Cop" Soundtrack Glenn Frey'
Searching for "It's a Sin - 2001 Remaster" by Pet Shop Boys...
No results found for: 'It's a Sin - 2001 Remaster Pet Shop Boys'
Searching for "All Night Long (All Night) - Single Version" by Lionel Richie...
Done.
Searching for "Jump (Original Mix)" by The Pointer Sisters...
Done.
Searching for "Another Day in Paradise - 2016 R

No results found for: 'Aquarius/Let The Sunshine In (The Flesh Failures) [Remastered] The 5th Dimension'
Searching for "Moon River (From "Breakfast at Tiffany's")" by Andy Williams...
No results found for: 'Moon River (From "Breakfast at Tiffany's") Andy Williams'
Searching for "House of the Rising Sun" by The Animals...
Done.
Searching for "Living Doll - 2003 Remaster" by Cliff Richard & The Drifters...
No results found for: 'Living Doll - 2003 Remaster Cliff Richard & The Drifters'
Searching for "Poor Little Fool - Remastered" by Ricky Nelson...
Done.
Searching for "It's Not for Me to Say - From the MGM Film "Lizzie"" by Johnny Mathis...
No results found for: 'It's Not for Me to Say - From the MGM Film "Lizzie" Johnny Mathis'
Searching for "Oh Boy! - Overdubbed Version" by Buddy Holly...
No results found for: 'Oh Boy! - Overdubbed Version Buddy Holly'
Searching for "Donna - Single Version" by Ritchie Valens...
No results found for: 'Donna - Single Version Ritchie Valens'
Searching fo

...

No results found for: 'Adam Lambert Quoting Dr. Seuss Various Artists'
Searching for "Revelation (From "Boy Erased")" by Troye Sivan...
Done.
Searching for "Better Now - Recorded At Spotify Studios NYC" by Troye Sivan...
Done.
Searching for "Rufus Wainwright Spotify Pride Intro" by Various Artists...
No results found for: 'Rufus Wainwright Spotify Pride Intro Various Artists'
Searching for "Cre Spoda" by Klaus Nomi...
No results found for: 'Cre Spoda Klaus Nomi'
Searching for "Rufus Wainwright on Pride" by Various Artists...
No results found for: 'Rufus Wainwright on Pride Various Artists'
Searching for "Cabaret: Married" by John Kander...
No results found for: 'Cabaret: Married John Kander'
Searching for "In Between the Heartaches - Ichiban" by Dionne Warwick...
No results found for: 'In Between the Heartaches - Ichiban Dionne Warwick'
Searching for "Kathy Griffin: Intro - #PressPlayForPride" by Various Artists...
No results found for: 'Kathy Griffin: Intro - #PressPlayForPride Variou

Done.
Searching for "Can't Buy Me Love - Remastered" by The Beatles...
Done.
Searching for "Get Back - Remastered" by The Beatles...
Done.
Searching for "Sgt. Pepper's Lonely Hearts Club Band - Remastered 2009" by The Beatles...
Done.
Searching for "When I'm Sixty Four - Remastered 2009" by The Beatles...
Done.
Searching for "Yellow Submarine - Remastered" by The Beatles...
Done.
Searching for "All You Need Is Love - Remastered" by The Beatles...
Done.
Searching for "Back In The U.S.S.R. - Remastered 2009" by The Beatles...
No results found for: 'Back In The U.S.S.R. - Remastered 2009 The Beatles'
Searching for "Michelle - Remastered" by The Beatles...
Done.
Searching for "I Should Have Known Better - Remastered" by The Beatles...
Done.
Searching for "Across The Universe - Remastered" by The Beatles...
Done.
Searching for "A Day In The Life - Remastered 2009" by The Beatles...
Done.
Searching for "If I Fell - Remastered" by The Beatles...
Done.
Searching for "I Feel Fine - Remastered" 

Done.
Searching for "Wild Honey Pie - Remastered 2009" by The Beatles...
No results found for: 'Wild Honey Pie - Remastered 2009 The Beatles'
Searching for "Being For The Benefit Of Mr. Kite! - Remastered 2009" by The Beatles...
No results found for: 'Being For The Benefit Of Mr. Kite! - Remastered 2009 The Beatles'
Searching for "Carry That Weight - Remastered" by The Beatles...
Done.
Searching for "Mother Nature's Son - Remastered 2009" by The Beatles...
Done.
Searching for "Sgt. Pepper's Lonely Hearts Club Band - Reprise / Remastered 2009" by The Beatles...
No results found for: 'Sgt. Pepper's Lonely Hearts Club Band - Reprise / Remastered 2009 The Beatles'
Searching for "Her Majesty - Remastered" by The Beatles...
Done.
Searching for "Here Comes The Sun - Remastered 2009" by The Beatles...
Done.
Searching for "She Came In Through The Bathroom Window - Remastered" by The Beatles...
Done.
Searching for "It's Only Love - Remastered" by The Beatles...
Done.
Searching for "She Said She 

Done.
Searching for "Don't Let the Sun Catch You Crying (Main) - Mono" by Gerry & The Pacemakers...
No results found for: 'Don't Let the Sun Catch You Crying (Main) - Mono Gerry & The Pacemakers'
Searching for "A Day In The Life - Remastered" by The Beatles...
Done.
Searching for "It's for You - 2003 Remaster" by Cilla Black...
No results found for: 'It's for You - 2003 Remaster Cilla Black'
Searching for "Icarus" by Paul Winter...
Done.
Searching for "Diamond Dust" by Jeff Beck...
Done.
Searching for "Being For The Benefit Of Mr. Kite! - Remastered" by The Beatles...
No results found for: 'Being For The Benefit Of Mr. Kite! - Remastered The Beatles'
Searching for "Elizabethan Serenade - 2003 Remaster" by Ron Goodwin...
No results found for: 'Elizabethan Serenade - 2003 Remaster Ron Goodwin'
Searching for "Portrait Of My Love - Remastered 2010" by Matt Monro...
No results found for: 'Portrait Of My Love - Remastered 2010 Matt Monro'
Searching for "A Transport of Delight" by Flanders & 

Done.
Searching for "Lazy - Remastered 2012" by Deep Purple...
Done.
Searching for "Woman From Tokyo - Remastered 2000" by Deep Purple...
No results found for: 'Woman From Tokyo - Remastered 2000 Deep Purple'
Searching for "Mistreated - Remastered 2010" by Deep Purple...
No results found for: 'Mistreated - Remastered 2010 Deep Purple'
Searching for "Space Truckin' - Remastered 2012" by Deep Purple...
No results found for: 'Space Truckin' - Remastered 2012 Deep Purple'
Searching for "Sail Away - Remastered 2007" by Deep Purple...
Done.
Searching for "Might Just Take Your Life - Remastered 2005" by Deep Purple...
Done.
Searching for "Demon's Eye - 1996 Remaster" by Deep Purple...
Done.
Searching for "Strange Kind of Woman - 2002 Remastered Version" by Deep Purple...
Done.
Searching for "When A Blind Man Cries - Non Album B-Side;2012 - Remaster" by Deep Purple...
No results found for: 'When A Blind Man Cries - Non Album B-Side;2012 - Remaster Deep Purple'
Searching for "Soldier Of Fortune

In [153]:
dups.sort_values('lyrics')

,artist,featuring,title,track_id,popularity,explicit,release_year,lyrics
18938,The Beatles,NaN,Penny Lane,5RStjc42UAYI2NMY3cYpgz,0,False,1967,"""I promis'd you an account of what befel me, and am now ..."
18996,The Beatles,NaN,The Long And Winding Road,2rG2c14sIgzyIRdbw3edaL,0,False,1970,"""I promis'd you an account of what befel me, and am now ..."
19224,The Beatles,NaN,Dig A Pony,3eMeNJhwxiecXnSYy2NhfY,0,False,1970,"""I promis'd you an account of what befel me, and am now ..."
18941,The Beatles,NaN,Oh! Darling,59CLXQLZKxRPzMW8S8Gt9N,0,False,1969,"""I promis'd you an account of what befel me, and am now ..."
780,Sumsar,NaN,Amorous Rebound,0JYn3juUc1lZ4kFeGey1kI,60,False,2019,"""O lamentable ruins of the ill-fated Nicosia,[76] still ..."
...,...,...,...,...,...,...,...,...
20279,Jamiroquai,NaN,Too Young to Die,5aifJSSMQwlLcIjAAqgXNV,0,False,2006,NaN
20293,Jamiroquai,NaN,Radio,4aDIUdxQJc4QM9VKQF1xzQ,0,False,2006,NaN
20299,Jamiroquai,NaN,High Times,4LGZzACsdcdtx9FxIukGsS,0,False,1996,NaN
20305,Jamiroquai,NaN,Runaway,6O95QIeonGQReMyDepOvpK,0,False,2006,NaN


In [169]:
# fixed.to_csv('./data/corrected_lyrics_cp1', index= False)

In [ ]:
df.drop([18560,18396, 18142, 24055, 26249, 26340, 21250, 19292, 23153, 23015, 22335, 27349, 20730, 26269, 27047, 23754, 18408, 23305, 25900, 22748, 26535, 24798, 19535, 23311, 26047, 19283, 20975, 23008, 25515, 26945, 26928, 27289, 20338, 20875, 19101, 19320, 25317, 20617, 20507, 18885, 24323, 23382, 19584, 19872, 23693, 19011, 25536, 26539, 18238, 26941, 21254, 23703, 24029, 23987, 19540, 19522, 19574, 20777, 20453, 18882, 26938, 25295, 19037, 19509, 19891, 23076, 23315, 20335, 25282, 18679, 19044, 19030, 19045, 23005, 19585, 26284, 25897, 26292, 18412, 23738, 25934], inplace=True)

In [ ]:
def fill_lyrics(artist, title, track_id):
    genius = lyricsgenius.Genius("2AaY5B5Iylvf3X2oQl_4Jmk74pSZ-J8VqcboMk1PO_y7N8wlN83D9a85uhNh49Id")
    song = genius.search_song(title, artist)
    df.loc[df['track_id'] == track_id, 'lyrics'] = song.lyrics
    

In [164]:
fixed[fixed['title'].str.contains('The Long And Winding Road')]

,artist,featuring,title,track_id,popularity,explicit,release_year,lyrics
18996,The Beatles,NaN,The Long And Winding Road - Remastered,2rG2c14sIgzyIRdbw3edaL,0,False,1970,"""I promis'd you an account of what befel me, and am now ..."
19055,The Beatles,NaN,The Long And Winding Road - Remastered 2015,0Oroc0HXQaxs8ONgI7dLnw,0,False,2000,NaN


7712 rows to 6962 rows of duplicates with 4734 as NaN

### Dropping Null Lyrics
Since this is an NLP project, if the lyrics column have not gathered any values (after running through the pull request twice), it is time to drop these songs. There may be some 'duplicates' based on lyrics, I will keep those, because it is the artist who decides to sing their own name, I may listen to the song from one artist, but not the other.

In [141]:
dups['lyrics'].isnull().sum()

4734

In [ ]:
df

### Cleaning up the Lyrics
The lyrics column has some obscure character such as \n and \ within the strings that need to be removed.
<br> The lyrics also contain [Verse] sections that list which artist is singing that part. This needs to be removed because it isn't part of the song and it will throw my models off, when it looks for the artist in the lyrics and creating my skip column.</br>

In [165]:
df['lyricss'] = df['lyrics'].str.replace('\n', ' ').str.replace("\'", "")

In [167]:
df['lyricss'] = df['lyrics'].replace(r'\[[^)]*\]','')

In [168]:
df

,artists,featuring,title,track_id,popularity,explicit,release_year,lyricss,lyrics
0,Shawn Mendes,,Wonder,5KCbr5ndeby4y4ggthdiAb,0,False,2020,[Verse 1]\nI wonder if I'm being real\nDo I speak my tru...,NaN
1,Justin Bieber,Chance The Rapper,Holy,5u1n1kITHCxxp8twBcZxWy,92,False,2020,[Verse 1: Justin Bieber]\nI hear a lot about sinners\nDo...,NaN
2,24kGoldn,Iann Dior,Mood,3tjFYV6RSFtuktYl3ZtYcq,100,True,2020,"[Intro: 24kGoldn]\nOh-oh-oh\nYeah, yeah, yeah, yeah (Yea...",NaN
3,Internet Money,,Lemonade,02kDW379Yfd5PzW5A6vuGt,93,True,2020,"[Pre-Chorus: Don Toliver]\nXanny bars, suicide door, bra...",NaN
4,BLACKPINK,Cardi B,Bet You Wanna,1hPkiovjTqiJAJen4uyNRg,0,False,2020,"[Intro: Cardi B]\nBLACKPINK\nCardi\n\n[Verse 1: Rosé, Li...",NaN
...,...,...,...,...,...,...,...,...,...
20506,Lady Gaga,,Judas,4yQmDtjax2Ye9wY8EPAO1x,58,False,2011,[Intro]\nWhoa-whoa-whoa-whoa-whoa\nI’m in love with Juda...,NaN
20509,Lady Gaga,,Paparazzi,02XnQdf7sipaKBBHixz3Zp,70,False,2008,[Verse 1]\nWe are the crowd\nWe're c-coming out\nGot my ...,NaN
20510,Lady Gaga,,LoveGame,0eH2eHURaXUP15D8gQlfjx,63,False,2008,"[Intro]\nLet's have some fun, this beat is sick\nI wanna...",NaN
20511,Lady Gaga,,Perfect Illusion,2KZSAE8Q2czVxKAzEuxbNu,50,False,2016,[Verse 1]\nTryin' to get control\nPressure's takin' its ...,NaN
